In [ ]:
import os
import gdown
import shutil

def check_download(name, folder, url):   
    fullname = folder+'/'+base_name+name+'.ms'          
    if not os.path.exists(fullname):
        gdown.download(url, fullname+'.zip')
        shutil.unpack_archive(filename=fullname+'.zip', extract_dir=folder)

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)
        
def build_folder_structure(dataname, resultname):
    create_folder(dataname)
    create_folder(resultname)
    

In [ ]:
# Build folder structure

datafolder = 'data'
resultsfolder = './results'
build_folder_structure(datafolder, resultsfolder)

In [ ]:
# Check if data is there or download it if needed
base_name = 'ea25_cal_small_spw1_4_60_ea04_'
before_url='https://drive.google.com/u/0/uc?id=1-v1foZ4G-kHTOS2idylx-3S4snKgRHmM&export=download'
after_url = 'https://drive.google.com/u/0/uc?id=1PmWvPA0rUtAfegVu9wOb4AGJXiQIp3Cp&export=download'

check_download('before', datafolder, before_url)
check_download('after', datafolder, after_url)

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

log_parms = {'log_level':'DEBUG'}

client = astrohack_local_client(cores=2,memory_limit='8GB',log_parms=log_parms)

In [ ]:
from astrohack.extract_holog import extract_holog
scans=[8,9,10,12,13,14,16,17,18,23,24,25,27,28,29,31,32,33,38,39,40,42,43,44,46,47,48,53,54,55,57]
holog_obs_description = {'map_0' :{'scans':scans,'ant':{'ea25':['ea04']}}} #change 0 -> map_0
holog_obs_description['ddi'] = [0]

beforems = datafolder+'/'+ base_name + 'before.ms'
beforeholog = resultsfolder+'/before.holog.zarr'
afterms =  datafolder+'/'+ base_name + 'after.ms'
afterholog = resultsfolder+'/after.holog.zarr'

extract_holog(
    ms_name=beforems, 
    holog_name=beforeholog, 
    holog_obs_dict=holog_obs_description,
    data_col='CORRECTED_DATA',
    parallel=True,
    overwrite=True,
)

extract_holog(
    ms_name=afterms, 
    holog_name=afterholog, 
    holog_obs_dict=holog_obs_description,
    data_col='CORRECTED_DATA',
    parallel=True,
    overwrite=True,
)

In [ ]:
import numpy as np
from astrohack import holog

cell_size = np.array([-0.0006442, 0.0006442])
grid_size = np.array([31, 31])
grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
reference_scaling_frequency = None #1.5*10**10 #None #In Hz
scan_average = True

holog(holog_name=beforeholog, padding_factor=50, 
      grid_size=grid_size, cell_size=cell_size, grid_interpolation_mode=grid_interpolation_mode,
      chan_average = chan_average,
      reference_scaling_frequency=reference_scaling_frequency,
      scan_average = scan_average,
      overwrite=True,
      phase_fit=True,
      apply_mask=True,
      to_stokes=True,
      parallel=True
     )

holog(holog_name=afterholog, padding_factor=50, 
      grid_size=grid_size, cell_size=cell_size, grid_interpolation_mode=grid_interpolation_mode,
      chan_average = chan_average,
      reference_scaling_frequency=reference_scaling_frequency,
      scan_average = scan_average,
      overwrite=True,
      phase_fit=True,
      apply_mask=True,
      to_stokes=True,
      parallel=True
     )

In [ ]:
import xarray as xr
#%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import constants
import matplotlib.patches as patches

beforeimage = resultsfolder+'/before.image.zarr'
afterimage = resultsfolder+'/after.image.zarr'

plt.close('all')

chan = 0

ds = xr.open_zarr(afterimage+'/ant_ea25/ddi_0')
wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])
#ax.add_patch(circle)
plt.show()

ds

In [ ]:
from astrohack.panel import panel
panel_model = 'rigid'

before_panel = panel(
    image_name=beforeimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)
after_panel = panel(
    image_name=afterimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
beforescrews = resultsfolder+'/before-screws.txt'
afterscrews = resultsfolder+'/after-screws.txt'
before_ant = before_panel.get_antenna('ant_ea25', 'ddi_0')
after_ant = after_panel.get_antenna('ant_ea25', 'ddi_0')

before_ant.export_screws(beforescrews, unit="mils")
after_ant.export_screws(afterscrews, unit="mils")

In [ ]:
import numpy as np
panellist = ['3- 4', '5-27', '5-37', '5-38']
expected  = np.array([-100, 75, 0, 150])

ref_means = np.array([-112.2025,   73.34  ,   -1.4025,  139.135 ])
ref_perce = (ref_means-expected)/expected*100.

def extract_panel_data(fname, panellist):
    data = open(fname, 'r')
    panels = []
    for line in data:
        wrds = line.split('  ')
        if wrds[0] in panellist:
            nmbrs = line.split()
            panel = [0, 0, 0, 0]
            for i in range(4):
                panel[-1-i] = float(nmbrs[-1-i])
            panels.append(panel)
    return np.array(panels)
    
def compare_before_after(panellist, expected): 
    after = extract_panel_data(afterscrews, panellist)
    before = extract_panel_data(beforescrews, panellist)
    diffs = after-before
    means = np.mean(diffs, axis=1)
    percent = (means-expected)/expected*100.
    return means, percent

def test_improvement(pname, diff, unit):
    abso = np.abs(diff)
    if abso > 1e-13:
        if diff < 0:
            print('Panel {0:s} has improved by {1:.20f} {2:s}'.format(pname, abso, unit))
        else:
            print('Panel {0:s} is worse by {1:.20f} {2:s}'.format(pname, abso, unit))
    else:
        print('Panel {0:s} is unchanged'.format(pname))

def is_it_an_improvement(means, percent):
    sumcomp = np.sum(ref_means-means)
    if abs(sumcomp) < 1e-13:
        print('No changes!')
    else:
        print('Results have changed')
        for i in range(4):
            pname = panellist[i]
            if expected[i] == 0:
                diff = np.abs(means[i]-ref_means[i])
                test_improvement(pname, diff, 'mils')
            else:
                diff = np.abs(percent[i]) - np.abs(ref_perce[i])
                test_improvement(pname, diff, '%')


                
means, percent = compare_before_after(panellist, expected)
is_it_an_improvement(means, percent)